In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets 
from torchvision import transforms

from tqdm import tqdm, tqdm_notebook

In [2]:
batch_size = 128
device = torch.device('cuda')

In [3]:
def train(model, device, train_loader, optimizer, criterion, n_epochs, log_interval):
    model.train()
    for epoch in tqdm_notebook(range(n_epochs), desc = 'Epochs'):
        for batch_idx, (X, y) in tqdm_notebook(enumerate(train_loader), total = len(train_loader), desc = 'Batches', leave = False):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(X)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print(f'Train epoch {epoch}: [{batch_idx * len(X):5d}/{len(train_loader.dataset):5d}] Loss: {loss.item():7.4f}')

In [4]:
def evaluate(model, device, test_loader, criterion):
    model.eval()
    test_set_size = len(test_loader.dataset)
    correct_answers = 0
    sum_loss = 0
    with torch.no_grad():
        for X, y in tqdm_notebook(test_loader):
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            class_pred = y_pred.argmax(dim = 1)
            correct_answers += (y == class_pred).sum().item()
            sum_loss += criterion(y_pred, y).item()
    accuracy = correct_answers / test_set_size
    average_loss = sum_loss / len(test_loader)
    
    return accuracy, average_loss

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1  , 64 , 3, padding = 1)
        self.conv2 = nn.Conv2d(64 , 64 , 3, padding = 1)
        self.conv3 = nn.Conv2d(64 , 128, 3, padding = 1)
        self.conv4 = nn.Conv2d(128, 128, 3, padding = 1)
        self.fc1   = nn.Linear(128 * 7 * 7, 256)
        self.fc2   = nn.Linear(256        , 256)
        self.fc3   = nn.Linear(256        , 10)
        
    def forward(self, x):
        x = F.dropout(F.relu(self.conv1(x)), 0.1)
        x = F.dropout(F.relu(self.conv2(x)), 0.1)
        x = F.max_pool2d(x, kernel_size = (2, 2))
        x = F.dropout(F.relu(self.conv3(x)), 0.1)
        x = F.dropout(F.relu(self.conv4(x)), 0.1)
        x = F.max_pool2d(x, kernel_size = (2, 2))
        x = x.view(-1, 128 * 7 * 7)
        x = F.dropout(F.relu(self.fc1(x)), 0.1)
        x = F.dropout(F.relu(self.fc2(x)), 0.1)
        x = self.fc3(x)
        
        return x

In [6]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        '../data',
        train = True,
        download = True,
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ]
        )
    ),
    shuffle = True,
    batch_size = batch_size
)

In [7]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        '../data',
        train = False,
        download = True,
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ]
        )
    ),
    shuffle = False,
    batch_size = 2 * batch_size
)

In [8]:
cnn       = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters())

In [9]:
train(cnn, device, train_loader, optimizer, criterion, 12, len(train_loader) // 5)

Train epoch 0: [    0/60000] Loss:  2.2994
Train epoch 0: [11904/60000] Loss:  0.2093
Train epoch 0: [23808/60000] Loss:  0.1711
Train epoch 0: [35712/60000] Loss:  0.0681
Train epoch 0: [47616/60000] Loss:  0.0233
Train epoch 0: [59520/60000] Loss:  0.0607


Train epoch 1: [    0/60000] Loss:  0.0178
Train epoch 1: [11904/60000] Loss:  0.0129
Train epoch 1: [23808/60000] Loss:  0.0215
Train epoch 1: [35712/60000] Loss:  0.0931
Train epoch 1: [47616/60000] Loss:  0.0701
Train epoch 1: [59520/60000] Loss:  0.0234


Train epoch 2: [    0/60000] Loss:  0.0377
Train epoch 2: [11904/60000] Loss:  0.0210
Train epoch 2: [23808/60000] Loss:  0.0084
Train epoch 2: [35712/60000] Loss:  0.0086
Train epoch 2: [47616/60000] Loss:  0.0164
Train epoch 2: [59520/60000] Loss:  0.0294


Train epoch 3: [    0/60000] Loss:  0.0110
Train epoch 3: [11904/60000] Loss:  0.0155
Train epoch 3: [23808/60000] Loss:  0.0044
Train epoch 3: [35712/60000] Loss:  0.0139
Train epoch 3: [47616/60000] Loss:  0.0537
Train epoch 3: [59520/60000] Loss:  0.0420


Train epoch 4: [    0/60000] Loss:  0.0042
Train epoch 4: [11904/60000] Loss:  0.0668
Train epoch 4: [23808/60000] Loss:  0.0033
Train epoch 4: [35712/60000] Loss:  0.0029
Train epoch 4: [47616/60000] Loss:  0.0028
Train epoch 4: [59520/60000] Loss:  0.0054


...

Train epoch 7: [    0/60000] Loss:  0.0015
Train epoch 7: [11904/60000] Loss:  0.0073
Train epoch 7: [23808/60000] Loss:  0.0163
Train epoch 7: [35712/60000] Loss:  0.0015
Train epoch 7: [47616/60000] Loss:  0.0100
Train epoch 7: [59520/60000] Loss:  0.0020


Train epoch 8: [    0/60000] Loss:  0.0005
Train epoch 8: [11904/60000] Loss:  0.0643
Train epoch 8: [23808/60000] Loss:  0.0748
Train epoch 8: [35712/60000] Loss:  0.0060
Train epoch 8: [47616/60000] Loss:  0.0395
Train epoch 8: [59520/60000] Loss:  0.0004


Train epoch 9: [    0/60000] Loss:  0.0118
Train epoch 9: [11904/60000] Loss:  0.0019
Train epoch 9: [23808/60000] Loss:  0.0004
Train epoch 9: [35712/60000] Loss:  0.0013
Train epoch 9: [47616/60000] Loss:  0.0060
Train epoch 9: [59520/60000] Loss:  0.0076


Train epoch 10: [    0/60000] Loss:  0.0003
Train epoch 10: [11904/60000] Loss:  0.0382
Train epoch 10: [23808/60000] Loss:  0.0022
Train epoch 10: [35712/60000] Loss:  0.0075
Train epoch 10: [47616/60000] Loss:  0.0197
Train epoch 10: [59520/60000] Loss:  0.0232


Train epoch 11: [    0/60000] Loss:  0.0001
Train epoch 11: [11904/60000] Loss:  0.0001
Train epoch 11: [23808/60000] Loss:  0.0029
Train epoch 11: [35712/60000] Loss:  0.0240
Train epoch 11: [47616/60000] Loss:  0.0109
Train epoch 11: [59520/60000] Loss:  0.0395



In [10]:
accuracy, avg_loss = evaluate(cnn, device, train_loader, criterion)
print(f'[Train] Accuracy: {100 * accuracy:5.2f}%, loss: {avg_loss:7.4f}')
accuracy, avg_loss = evaluate(cnn, device, test_loader, criterion)
print(f'[Test] Accuracy: {100 * accuracy:5.2f}%, loss: {avg_loss:7.4f}')


[Train] Accuracy: 99.67%, loss:  0.0114



[Test] Accuracy: 99.05%, loss:  0.0356
